In [1]:
import pandas as pd
from utils import binarize_prediction
from inference import get_classes
import numpy as np
from sklearn.metrics import fbeta_score

In [8]:
#df0 = pd.read_csv('./savings/probs_sresnet50.0.csv').set_index('id')
df1 = pd.read_csv('./savings/probs_seresnext.1.0_1.csv').set_index('id')
df2 = pd.read_csv('./savings/probs_seresnext.2.0_1.csv').set_index('id')
df3 = pd.read_csv('./savings/probs_seresnext.3.0_1.csv').set_index('id')
df4 = pd.read_csv('./savings/probs_seresnext.4.0_1.csv').set_index('id')

df0_resnet = pd.read_csv('./savings/probs_resnet50.0.csv').set_index('id')
df1_resnet = pd.read_csv('./savings/probs_resnet50.1.csv').set_index('id')
df2_resnet = pd.read_csv('./savings/probs_resnet50.2.csv').set_index('id')
df3_resnet = pd.read_csv('./savings/probs_resnet50.3.csv').set_index('id')
df4_resnet = pd.read_csv('./savings/probs_resnet50.4.csv').set_index('id')

In [18]:
def submission_file(each_class_threshold=False):
    df_ens = df1 + df2 + df3 + df4 + df0_resnet + df1_resnet + df2_resnet + df3_resnet + df4_resnet
    df_ens[:] = df_ens.values/9
    
    if each_class_threshold:
        ths = get_thresholds()
        out = binarize_prediction(df_ens.values, ths)
    else:
        out = binarize_prediction(df_ens.values, 0.1)
    df_ens[:] = out

    df_ens = df_ens.apply(get_classes, axis=1)
    df_ens.name = 'attribute_ids'

    df_ens.to_csv('./submissions/cv9_seresnext_resnet.csv', header=True)
    return df_ens

In [19]:
df = submission_file()

In [5]:
def binarize_prediction(probabilities, threshold: float, argsorted=None,
                        min_labels=1, max_labels=10):
    N_CLASSES = 1103
    assert probabilities.shape[1] == N_CLASSES
    if argsorted is None:
        argsorted = probabilities.argsort(axis=1)

    def _make_mask(argsorted, top_n: int):
        mask = np.zeros_like(argsorted, dtype=np.uint8)
        col_indices = argsorted[:, -top_n:].reshape(-1)
        row_indices = [i // top_n for i in range(len(col_indices))]
        mask[row_indices, col_indices] = 1
        return mask

    max_mask = _make_mask(argsorted, max_labels)
    min_mask = _make_mask(argsorted, min_labels)
    prob_mask = probabilities > threshold
    return (max_mask & prob_mask) | min_mask

def get_score(targets, y_pred):
    return fbeta_score(targets, y_pred, beta = 2)

In [7]:
def get_thresholds():
    predictions = np.load('./savings/predictions.npy')
    targets = np.load('./savings/targets.npy')
    ths = []
    N_CLASSES = 1103
    for i in range(N_CLASSES):
        current_scores = []
        for threshold in np.linspace(0.1, 1, 10):
            tgt = np.expand_dims(targets[:, i], 1)
            prd = np.expand_dims(predictions[:, i], 1)
            current_scores.append((get_score(tgt, (prd > threshold).astype('int')),threshold))

        ths.append(max(current_scores, key = lambda x: x[0])[1])
    ths = np.array(ths)
    ths = np.expand_dims(ths, 0)
    return ths